In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import time
from os import listdir

In [ ]:
def load_data(dir_list, image_size):

    X = []
    y = []
    image_width, image_height = image_size
    
    for directory in dir_list:
        for filename in listdir(directory):
            
            image = cv2.imread(directory + '/' + filename)
            image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
            image = image / 255.
            X.append(image)
            if directory[-3:] == 'ari':
                y.append([1])
            else:
                y.append([0])
                
    X = np.array(X)
    y = np.array(y)

    X, y = shuffle(X, y)
    
    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')
    
    return X, y

In [ ]:
dataset = '/content/drive/MyDrive/Cars/dataset/'

dataset_yes = dataset + 'Ferrari' 
dataset_no = dataset + 'Honda'

IMG_WIDTH, IMG_HEIGHT = (240, 240)

X, y = load_data([dataset_yes, dataset_no], (IMG_WIDTH, IMG_HEIGHT))

In [ ]:
def split_data(X, y, test_size=0.1):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)
    
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = split_data(X, y, test_size=0.1)

In [ ]:
del X
del y

In [ ]:
def build_model(input_shape):

    X_input = Input(input_shape) 
    
    X = ZeroPadding2D((2, 2))(X_input) 

    X = Conv2D(32, (5, 5), strides = (1, 1), name = 'conv0')(X)
    X = Activation('relu')(X)

    X = MaxPooling2D((2, 2), name='max_pool0')(X)
    
    X = Conv2D(32, (5, 5), strides = (1, 1), name = 'conv1')(X)
    X = Activation('relu')(X)

    X = MaxPooling2D((2, 2), name='max_pool1')(X)

    X = Flatten()(X) 

    X = Dense(1, activation='sigmoid', name='fc')(X)
    
    model = Model(inputs = X_input, outputs = X, name='VehicleDetectionModel')
    
    return model

In [ ]:
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

In [ ]:
model = build_model(IMG_SHAPE)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:

filepath="cnn-parameters-improvement-{epoch:02d}-{val_accuracy:.2f}"

checkpoint = ModelCheckpoint("{}.model".format(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'))

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)#,min_delta=0)

In [ ]:
model.fit(x=X_train, y=y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test), callbacks=[checkpoint,callback])

In [ ]:
def load_data1(file_path,image_size):
    
    X2 = []
    
    image_width, image_height = image_size
    print(file_path)
    image = cv2.imread(file_path)
    image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
    image = image / 255.
    X2.append(image)
    
    X2 = np.array(X2)
    
    print(f'Number of examples is: {len(X2)}')
    print(f'X shape is: {X2.shape}')
    
    return X2
    

In [ ]:
file_path='/content/drive/MyDrive/Cars/dataset/Honda/010352.jpg'
img=load_data1(file_path,(IMG_WIDTH, IMG_HEIGHT))

In [ ]:

ss=model.predict(img)

if ss[0][0]>0.5:
    output="Ferrari"
else:
    output="Honda"
        
print(output)

In [ ]:
best_model = load_model(filepath='cnn-parameters-improvement-01-0.89.model')

ss=best_model.predict(img)


if ss[0][0]>0.5:
    output="Ferrari"
else:
    output="Honda"
        
print(output)